In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/END

/content/drive/MyDrive/END


In [ ]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from tqdm.notebook import tqdm
import torchtext
import spacy
from torchtext.data import Field
import torch
import torch.nn as nn
from torchtext.data import BucketIterator
import torch.optim as optim
import time
import random
import math
from torchtext import data

In [ ]:
train_data = pd.read_json("CoQA/coqa-train-v1.0.json")
test_data = pd.read_json("CoQA/coqa-dev-v1.0.json")

In [ ]:
train_data.head()

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."


In [ ]:
train_data['data'][0]['story']

'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of t

In [ ]:
train_data['data'][0]['questions']

[{'input_text': 'When was the Vat formally opened?', 'turn_id': 1},
 {'input_text': 'what is the library for?', 'turn_id': 2},
 {'input_text': 'for what subjects?', 'turn_id': 3},
 {'input_text': 'and?', 'turn_id': 4},
 {'input_text': 'what was started in 2014?', 'turn_id': 5},
 {'input_text': 'how do scholars divide the library?', 'turn_id': 6},
 {'input_text': 'how many?', 'turn_id': 7},
 {'input_text': 'what is the official name of the Vat?', 'turn_id': 8},
 {'input_text': 'where is it?', 'turn_id': 9},
 {'input_text': 'how many printed books does it contain?', 'turn_id': 10},
 {'input_text': 'when were the Secret Archives moved from the rest of the library?',
  'turn_id': 11},
 {'input_text': 'how many items are in this secret collection?',
  'turn_id': 12},
 {'input_text': 'Can anyone use this library?', 'turn_id': 13},
 {'input_text': 'what must be requested to view?', 'turn_id': 14},
 {'input_text': 'what must be requested in person or by mail?', 'turn_id': 15},
 {'input_text': 

In [ ]:
train_data['data'][0]['answers']

[{'input_text': 'It was formally established in 1475',
  'span_end': 179,
  'span_start': 151,
  'span_text': 'Formally established in 1475',
  'turn_id': 1},
 {'input_text': 'research',
  'span_end': 494,
  'span_start': 454,
  'span_text': 'he Vatican Library is a research library',
  'turn_id': 2},
 {'input_text': 'history, and law',
  'span_end': 511,
  'span_start': 457,
  'span_text': 'Vatican Library is a research library for history, law',
  'turn_id': 3},
 {'input_text': 'philosophy, science and theology',
  'span_end': 545,
  'span_start': 457,
  'span_text': 'Vatican Library is a research library for history, law, philosophy, science and theology',
  'turn_id': 4},
 {'input_text': 'a  project',
  'span_end': 879,
  'span_start': 769,
  'span_text': 'March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts',
  'turn_id': 5},
 {'input_text': 'into periods',
  'span_end': 1127,
  'span_start': 1048,
  'span_text': 'Scholars 

In [ ]:
len(train_data['data'][0]['questions']), len(train_data['data'][0]['answers'])

(20, 20)

In [ ]:
def make_dataframe(data):
  data_df = DataFrame()
  for i in tqdm(range(len(data)), total = len(data)):
      obs = data['data'][i]
      story = obs['story']
      pairs = len(obs['questions'])
      for pair in range(pairs):
          ques = obs['questions'][pair]
          ans = obs['answers'][pair]
          row_df = DataFrame({'story_id':i,'story':story, 'question':ques['input_text'], 
                              'answer':ans['input_text'], 'q_tid':ques['turn_id'],
                              'a_tid':ans['turn_id'], 'ans_start':ans['span_start'],
                              'ans_end': ans['span_end']}, index=[0])
          data_df = pd.concat([data_df, row_df], axis = 0)
  data_df.reset_index(drop = True, inplace = True)

  data_df['story'] = data_df['story'].apply(lambda x:x.strip(' '))
  data_df['ques_context'] = data_df.apply(lambda row:row['question'] + " " + row['story'], axis=1)
  data_df['sent_length'] = data_df['ques_context'].apply(lambda x:len(x))

  return data_df

In [ ]:
train_df = make_dataframe(train_data)
test_df = make_dataframe(test_data)

In [ ]:
nlp = spacy.load("en")

In [ ]:
for token in nlp(train_df['ques_context'][0]):
    print(token.text, "-->", token.dep_, "-->", token.pos_)

When --> advmod --> ADV
was --> aux --> AUX
the --> det --> DET
Vat --> nsubj --> PROPN
formally --> advmod --> ADV
opened --> ROOT --> VERB
? --> punct --> PUNCT
The --> det --> DET
Vatican --> compound --> PROPN
Apostolic --> compound --> PROPN
Library --> nsubj --> PROPN
( --> punct --> PUNCT
) --> punct --> PUNCT
, --> punct --> PUNCT
more --> advmod --> ADV
commonly --> advmod --> ADV
called --> parataxis --> VERB
the --> det --> DET
Vatican --> compound --> PROPN
Library --> oprd --> PROPN
or --> cc --> CCONJ
simply --> advmod --> ADV
the --> det --> DET
Vat --> conj --> NOUN
, --> punct --> PUNCT
is --> ROOT --> AUX
the --> det --> DET
library --> attr --> NOUN
of --> prep --> ADP
the --> det --> DET
Holy --> compound --> PROPN
See --> pobj --> PROPN
, --> punct --> PUNCT
located --> acl --> VERB
in --> prep --> ADP
Vatican --> compound --> PROPN
City --> pobj --> PROPN
. --> punct --> PUNCT
Formally --> advmod --> ADV
established --> ROOT --> VERB
in --> prep --> ADP
1475 --> p

In [ ]:
def tokenize(text):
  return [token.text for token in nlp.tokenizer(text)]

In [ ]:
# train_df.to_csv("COQA_train.csv", index = False)
train_df = pd.read_csv("COQA_train.csv")

In [ ]:
text_field = Field(
    sequential=True,
    tokenize=tokenize, 
    lower=True,
    init_token='<sos>', eos_token='<eos>'
)

In [ ]:
fields = {'ques_context' : ('ques_context', text_field),
          'answer':('answer', text_field)}

In [ ]:
text_dataset = torchtext.data.TabularDataset(path='COQA_train.csv', format='CSV', fields=fields)

In [ ]:
train, valid = text_dataset.split(split_ratio=[0.7, 0.3])

In [ ]:
len(train), len(valid)

(76053, 32594)

In [ ]:
MAX_VOCAB_SIZE = 20000
text_field.build_vocab(train, min_freq=2, max_size=MAX_VOCAB_SIZE)

In [ ]:
def get_example(data, example_number):
    print(f"Question and Context : \n\n{' '.join(data.examples[example_number].ques_context)}")
    print(f"\nAnswer : \n\n{' '.join(data.examples[example_number].answer)}")

In [ ]:
get_example(train, 0)

Question and Context : 

when was it anounced ? olympic torch relay planned route beijing , april 27 - -the beijing 2008 olympic torch relay will cover the greatest area and include the largest number of people . the plans were announced on thursday night . the planned route would last 130 days and travel 137 000 km . first nobel prize winner to donate to hope school chinanews , beijing , may 10 - -professor dannel chee tsui , in the usa , signed the agreement to donate 350 000yuan to his home town to build a hope primary school in china . bill gates receives tsinghua honorary doctorate beijing , april 20 - -bill gates , chairman of global it giant microsoft , received an honorary doctorate when he visited china 's famous tsinghua university on thursday . big shaolin kung fu center to be built in russia chinanews , beijing , april 27 - -china and russia have signed a letter of intent ( ) on the building of a shaolin kung fu center in russia . it will be the first of its kind in russia 

In [ ]:
get_example(valid, 0)

Question and Context : 

what did they want ? ( cnn ) -- a line of angry protesters waving signs and wearing scows formed a ring around the front entrance of the daily news ' headquarters . 

 they took turns at the bullhorn accusing the paper of everything from libel to genocide . they did n't bring a list of demands ; they were n't looking to negotiate . they had one goal : to shut the paper down forever . 

 " we 're going to march until the walls come down , " one shouted . 

 employees who would normally head out the revolving door to one of the lunch trucks along broad street developed a taste for cafeteria food that day . 

 not chuck stone . 

 stone , senior editor of the newspaper they had pledged to kill , walked out the front entrance and met their scows with a broad smile . picketers committed to the complete destruction of the daily news returned his smile or nodded in recognition as they passed him . a few even shook his hand . 

 i 'll never forget that scene . it was ,

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
BATCH_SIZE = 32

train_iterator, valid_iterator = BucketIterator.splits((train, valid), batch_size = BATCH_SIZE, sort_key = lambda x: len(x.ques_context), sort_within_batch = False, device = device)

In [ ]:
batch_ex = next(iter(train_iterator))
batch_ex.ques_context

tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [ 87,  50,  99,  ...,  50,  14,  50],
        [ 36,  14, 105,  ..., 448,  15,  16],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')

In [ ]:
batch_ex.ques_context.shape

torch.Size([842, 32])

In [ ]:
[text_field.vocab.itos[x] for x in batch_ex.ques_context[:, 0]]

['<sos>',
 'if',
 'they',
 'were',
 "n't",
 ',',
 'where',
 'would',
 'they',
 'have',
 'gone',
 '?',
 'chapter',
 'viii',
 '.',
 '\n\n',
 'the',
 'stealing',
 'of',
 'the',
 'horses',
 '.',
 '\n\n',
 '"',
 'go',
 'slow',
 ',',
 'men',
 ',',
 '"',
 'came',
 'softly',
 'from',
 'gilroy',
 '.',
 '"',
 'you',
 'know',
 'what',
 'kind',
 'of',
 'a',
 'man',
 'leeson',
 'is',
 '.',
 '"',
 '\n\n',
 '"',
 'reckon',
 'i',
 'do',
 'know',
 ',',
 '"',
 'came',
 'in',
 'a',
 'growl',
 'from',
 'fetter',
 '.',
 '"',
 'and',
 'i',
 "'ve",
 'got',
 'an',
 'account',
 'to',
 'settle',
 'with',
 'him',
 ',',
 'too',
 '.',
 '"',
 '\n\n',
 '"',
 'i',
 "'m",
 'pretty',
 'certain',
 'the',
 'boy',
 'is',
 'here',
 ',',
 '"',
 'went',
 'on',
 'the',
 'leader',
 '.',
 '"',
 'but',
 'we',
 'must',
 'make',
 'sure',
 'if',
 'the',
 'others',
 'are',
 'here',
 'too',
 ',',
 'or',
 'if',
 'they',
 'have',
 'ridden',
 'off',
 'to',
 'the',
 'fort',
 '.',
 'if',
 'they',
 'have',
 'gone',
 'to',
 'the',
 '<unk>',

In [ ]:
[text_field.vocab.itos[x] for x in batch_ex.answer[:, 0]]

['<sos>',
 'the',
 'fort',
 '<eos>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(text_field.vocab)
OUTPUT_DIM = len(text_field.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 128
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(20004, 128)
    (rnn): GRU(128, 128)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(20004, 128)
    (rnn): GRU(256, 128)
    (fc_out): Linear(in_features=384, out_features=20004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,069,860 trainable parameters


In [ ]:
optimizer = optim.AdamW(model.parameters())
PAD_IDX = text_field.vocab.stoi[text_field.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.ques_context
        trg = batch.answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.ques_context
            trg = batch.answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'COQA_model1.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 52s
	Train Loss: 5.222 | Train PPL: 185.377
	 Val. Loss: 5.535 |  Val. PPL: 253.438
Epoch: 02 | Time: 4m 52s
	Train Loss: 5.113 | Train PPL: 166.145
	 Val. Loss: 5.588 |  Val. PPL: 267.196
Epoch: 03 | Time: 4m 53s
	Train Loss: 4.980 | Train PPL: 145.511
	 Val. Loss: 5.430 |  Val. PPL: 228.082
Epoch: 04 | Time: 4m 56s
	Train Loss: 4.743 | Train PPL: 114.828
	 Val. Loss: 5.250 |  Val. PPL: 190.631
Epoch: 05 | Time: 4m 56s
	Train Loss: 4.499 | Train PPL:  89.886
	 Val. Loss: 5.151 |  Val. PPL: 172.639
Epoch: 06 | Time: 4m 55s
	Train Loss: 4.329 | Train PPL:  75.835
	 Val. Loss: 5.131 |  Val. PPL: 169.196
Epoch: 07 | Time: 4m 57s
	Train Loss: 4.194 | Train PPL:  66.270
	 Val. Loss: 5.106 |  Val. PPL: 164.986
Epoch: 08 | Time: 4m 58s
	Train Loss: 4.072 | Train PPL:  58.660
	 Val. Loss: 5.109 |  Val. PPL: 165.504
Epoch: 09 | Time: 4m 57s
	Train Loss: 3.943 | Train PPL:  51.581
	 Val. Loss: 5.121 |  Val. PPL: 167.468
Epoch: 10 | Time: 4m 57s
	Train Loss: 3.824 | Train PPL

In [ ]:
model.load_state_dict(torch.load('COQA_model1.pt'))

<All keys matched successfully>